# DATA Analysis

Different type of **Exploratory Data Analysis(EDA)**:
 * Univariate nongraphical
 * Multivariate nongraphical
 * Univariate graphical
 * Multivariate graphical

### Checking data by having a glimpse

In [ ]:
str(df)
dim(df)
describe(df)

# Unique values per column
lapply(df, function(x) length(unique(x)))

# Fast way to check NAs in each column of dataframe
colSums(is.na(df))
# or (as alternative)
sapply(df, function(x)sum(is.na(x)))
       
#Check the percentage of Missing values       
missing_values <- df %>% summarize_all(funs(sum(is.na(.))/n()))
missing_values <- gather(missing_values, key="feature", value="missing_pct")
       
## Another approach for getting a full vision of dataframe status
## is using funModeling library
library(funModeling)
df_status(df)
# Retrieves the distribution in a table and a plot and shows the distribution of absolute and relative numbers
freq(data=df, input = c('colname1','colname2',...))
# Full numerical profiling in one function automatically excludes non-numerical variables
profiling_num(data_world_wide)
# Plots the distribution of every numerical variable while automatically excluding the non-numerical ones       
plot_num(data_world_wide)       

### Data Query

In [ ]:
## Select exclusively (select all columns except mentioned)
select(df, -one_of(colname))

### Univariate Analysis

In [ ]:
# To count the occurrence of each value
table(df$colname)

# To get the percentage of occurence
prop.table(table(df$colname))
       
# In order to get most frequent value in a column of df
tail(names(sort(table(df$colname))), 1)

### Multivariate Analysis

In [ ]:
# Confusion table for two categorical variable in dataset
with(df, CrossTable(colname1, colname2))

# In order to show relation between two contiuous variable we can do the scatterplot
ggplot(data = df, aes(x = colname1 , y = colname2)) + geom_point()

# For a combination of continuous and categorical variables we can use boxplot
ggplot(data = df, aes(x = colname1 , y = colname2)) + geom_boxplot()

### Outliners

In [ ]:
# One approach for spoting outliners are the using geom_jitter
ggplot(df, aes(colname1, colname2)) + 
    geom_jitter(alpha = 0.2 ,size = 3)

# Data wrangling

#### Data reduction
If you use a large amount of data, it has bigger data size and needs more resources, moreover it may produce redundant results. In order to overcome such difficulties, we can use data reduction methods. hence, the storage efficiency will increase and at the same time we can minimize the data handling costs and will minimize the analysis time also. There are several types of data reduction:
 - **Filtering and sampling**
     - Moving average filtering
     - Savitzky-Golay filtering
     - High correlation filtering
     - Bayesian filtering
 - **Binned algorithm**
     - Equal-width binning
     - Equal-size binning
     - Optima binning
     - Multi-interval discretization binning
 - **Dimensionality reduction**
     - Principle Component Analysis (PCA)
     - Linear Discriminant Analysis (LDA)

# Data Cleaning

Useful Data minging/Cleaning tools for preparing messy data for analysis in R or python:
 - [OpenRefine](https://github.com/OpenRefine/OpenRefine)
 - [DataWrangler](http://vis.stanford.edu/wrangler/)
 

In [ ]:
# Renaming field names by using dplyr
df <- df %>% rename(
  newname1 = oldname1,
  newname2 = oldname2)

### Missing values

#### Removing NA's

In [ ]:
df <- na.omit(df)

#### Imputing NA's

In [ ]:
## First approach: simply replace NAs with mean of the column
df <- df %>%
    mutate( colname = ifelse(is.na(colname), 
                             mean(df$colname, na.rm=TRUE), 
                             colname))

# Use the most common value to replace NAs in the desired feature. 
# Imagine S was the most frequent 
# value for that field within the dataframe 
df$colname <- replace(df$colname, 
                      which(is.na(df$colname)), 
                      'S')

# One sublte way to filling null values in data set would be using
# prediction based on other explanatory variables. For example, 
# here we're looking for a countinous variable based on others.
exp_var_filler_model <- rpart(exp_var_with_many_NA ~ a_list_of_exp_vars,
                              data=df[!is.na(df$exp_var),], 
                              method="anova" #since we consider that exp_var as continous, otherwise we could use `class`)
df$exp_var[is.na(df$exp_var)] <- predict(exp_var_filler_model, 
                                         [is.na(df$exp_var),])
                              
## Third approach: using mice
library(mice)
simple_df_with_NA = df[c("NAcol1", "NAcol2", "col1", "col2")]
imputed = complete(mice(simple_df_with_NA))
df$NAcol1 = imputed$NAcol1
df$NAcol2 = imputed$NAcol2

# Data Sampling

In [ ]:
# Having a random sample dataframe from the original data by having only 3 row[for sake of example]
df_sample <- df[sample(nrow(df), 3), ]

# Using dplyr to have the same result as above
df_sample <- sample_n(df, size = 3)

# Data Manipulation (Feature Engineering)

### Feature Engineering
Feature engineering has been described as easily the most important factor in determining the success or failure of your predictive model. Feature engineering really boils down to the human element in machine learning. How much you understand the data, with your human intuition and creativity, can make the difference.

In [ ]:
df <- df %>%
    mutate( new_feature = case_when(raw_feature < 13 ~ "Lower.Range", 
                                    raw_feature >= 13 & raw_feature < 18 ~ "Mid.Range.1",
                                    raw_feature >= 18 & raw_feature < 60 ~ "Mid.Range.2",
                                    raw_feature >= 60 ~ "Upper.Range"))

# In order to combine same categorical data which has very little proportion from total we can use recode method
libray(car)
df$cat_var <- recode(df$cat_var, "c('group1', 'group2',...) = 'more_general_group'")
# Or simply
df3$cat_var[df$cat_var %in% c('group1','group2')] = 'more_general_group'

# Aggregate function

In [ ]:
# find the number of survivors for the different subsets
> aggregate(Survived ~ Child + Sex, data=train, FUN=sum)
  Child    Sex Survived
1     0 female      195
2     1 female       38
3     0   male       86
4     1   male       23

# To know the total number of people in each subset
> aggregate(Survived ~ Child + Sex, data=train, FUN=length)
  Child    Sex Survived
1     0 female      259
2     1 female       55
3     0   male      519
4     1   male       58

# To know the proportions
> aggregate(Survived ~ Child + Sex, data=train, 
            FUN=function(x) {sum(x)/length(x)})
  Child    Sex  Survived
1     0 female 0.7528958
2     1 female 0.6909091
3     0   male 0.1657033
4     1   male 0.3965517

# Correlation and Relationship
Perhaps the most standard correlation measure for numeric variables is the R statistic (or Pearson coefficient) which goes from 1 positive correlation to -1 negative correlation. A value around 0 implies no correlation.

Squaring this number returns the **R-squared** statistic (aka **R2**), which goes from 0 no correlation to 1 high correlation.

> R statistic is highly influenced by outliers and non-linear relationships. Highly recommended to **Plot** data as checking correlation as well as thinking about outliners before evaluating correlation.**


**<font color=blue>NOTE:</font>** Correlation can be measure better with Information Theory concepts. One of the many algorithms to measure correlation based on this is: MINE, acronym for: Maximal Information-based nonparametric exploration.

In [ ]:
## For evaluating correlation between two columns in dataset
cor(df$colname1, df$colname2)

## For getting the correlation table between all the columns of dataset
cor(df)

## By using funModeling package
library(funModeling)
correlation_table(data=df, target="colname")

## Using MINE algorithm
library(minerva)
res_mine <- df %>%
            select_if(is.numeric) %>%
            mine(use = 'pairwise.complete.obs')
res_mine$MIC

## MINE can also help us to profile time series regarding its non-monotonicity with MAS (maximum asymmetry score).
# MAS ~ 0 indicates monotonic function
# MAS ~ 1 indicates non-monotonic function (always up or down)
res_mine$MAS

###  Correlation on categorical variables

MINE -and many other algorithms- only work with numerical data. We need to do a data preparation trick, converting every categorical variable into flag (or dummy variable).

If the original categorical variable has 30 possible values, it will result in 30 new columns holding the value 0 or 1, when 1 represents the presence of that category in the row.

In [ ]:
library(caret)

# selecting just categorical column(s) which we're going to convert
df_cat=select(df, cat_colname1, cat_colname2, ...)
# it converts all categorical variables (factor and character for R) into numerical variables
# skipping the original so the data is ready to use
dmy = dummyVars(" ~ .", data = df_cat)
df_cat_explode = data.frame(predict(dmy, newdata = df_cat))
# Important: If you recieve this message `Error: Missing values present in input variable 'x'. 
# Consider using use = 'pairwise.complete.obs'.` is because data has missing values.
# Please don't omit NA without an impact analysis first, in this case it is not important. 
df_cat_explode = na.omit(df_cat_explode)
# compute the mic!
mine_res = mine(df_cat_explode)

### Visualizing the correlation

In [ ]:
## The corrplot package is a graphical display of a correlation matrix, confidence interval.
library(corrploy)

## simple case 1: having simple cor function as data provider
df %>%
  select_if(is.numeric) %>%
  cor(use="complete.obs") %>%
  corrplot.mixed(tl.cex=0.85)

# simple case 2: plotting MIC results by using circle method
corrplot(mine_res$MIC, method="circle",
         col=brewer.pal(n=10, name="PuOr"),
         # only display upper diagonal
         type="lower", 
         #label color, size and rotation
         tl.col="red", tl.cex = 0.9, tl.srt=90, 
         # dont print diagonal (var against itself)
         diag=FALSE, 
         # accept a any matrix, mic in this case (not a correlation
         #   element)
         is.corr = F)

## simple case 3: same as last case but by showing correlation values on plot
corrplot(mine_res$MIC, method="color",
         type="lower", number.cex=0.7,
         addCoef.col = "black", # Add coefficient of correlation
         tl.col="red", tl.srt=90, tl.cex = 0.9,
         diag=FALSE, is.corr = F)

# Regression
### Linear Regression

In [ ]:
fit <- lm(outcome ~ independent1 + independent2 + ..., data = train_df)
## Getting information about the model
summary(fit)

## Calculating Model R-squared
Model_R2 <- sum(fit$residuals^2)

## Calculating R-squared of out-of-same prediction
pred <- predict(fit, newdata=test_df)
SSE = sum((pred - test_df$outcome)^2)
# Consider the baseline is based on train dataset
SST = sum((test_df$outcome - mean(train_df$outcome))^2)
R2 = 1 - SSE/SST
RMSE = sqrt(SSE/nrow(test_df))
RMSE = sqrt(mean((pred - test_df$outcome)^2))

#### Simplicity the model
That is, a model with fewer variables is preferable to a model with many unnnecessary variables. 
Experiment with removing independent variables from the original model. We can use the significance of the coefficients to decide which variables to remove (remove the one with the <font color=red>**largest p-value**</font> first, or the one with the <font color=red>**t-value closest to zero**</font>), and to remove them one at a time (this is called "backwards variable selection"). This is important due to **multicollinearity** issues. <br/>
Removing one insignificant variable may make another previously insignificant variable become significant.

**NOTE:** When we remove insignificant variables, the "Multiple R-squared" will always be worse, but only slightly worse. This is due to the nature of a linear regression model. It is always possible for the regression model to make a coefficient zero, which would be the same as removing the variable from the model. The fact that the coefficient is not zero in the intial model means it must be helping the R-squared value, even if it is only a very small improvement. So when we force the variable to be removed, it will decrease the R-squared a little bit. However, this small decrease is worth it to have a simpler model.
On the contrary, when we remove insignificant variables, the "Adjusted R-squred" will frequently be better. This value accounts for the complexity of the model, and thus tends to increase as insignificant variables are removed, and decrease as insignificant variables are added.


In [ ]:
## If we don't have any idea about which independent variables 
# should be involved in formula, we can involve all and then using
# step()  method to simplify the complex model to simpler one.
messy_model <- lm(outcome ~ ., data = df)
simplified_model <- step(messy_model)
summary(simplified_model)

#### Unordered factors in regression models
To include unordered factors in a linear regression model, we define one level as the "reference level" and add a binary variable for each of the remaining levels. In this way, a factor with **n** levels is replaced by **n-1** binary variables. The reference level is typically selected to be the most frequently occurring level in the dataset.

As an example, consider the unordered factor variable `color`, with levels `red`, `green`, and `blue`. If "green" were the reference level, then we would add binary variables "colorred" and "colorblue" to a linear regression problem. All red examples would have `colorred=1` and `colorblue=0`. All blue examples would have `colorred=0` and `colorblue=1`. All green examples would have `colorred=0` and `colorblue=0`.

**Hint:** by default R selects the first level alphabetically as the reference level of our factor instead of the most common level. We can set the reference level of the factor by typing the following two lines in your R console:

In [ ]:
## Setting reference level in unordered factor variable in dataset
df$factor_colname = relevel(df$factor_colname, "MostFreqColName")

#### Modeling on skewed dependent variable

When handling a **skewed** dependent variable, it is often useful to predict the logarithm of the dependent variable instead of the dependent variable itself -- this prevents the small number of unusually large or small observations from having an undue influence on the sum of squared errors of predictive models, which can be computed in R using the log() function.

> fit <- lm(log(outcome) ~ colname, data=df)

However, the dependent variable in our model is log(outcome), so `Prediction` would contain predictions of the log(outcome) value. We are instead interested in obtaining predictions of the outcome value. We can convert from predictions of log(outcome) to predictions of outcome via exponentiation, or the `exp()` function. 

> Prediction = exp(predict(fit, newdata=test_df))

### Logistic Regression
Used for nonlinear dependent varaibles

In [ ]:
## Spliting data into Train and test set
library(caTools)
# First arg: the columns which we want to split on
# Second arg: the ratio of rows in train and test sets
splitIndex <- sample.split(df$outcome, SplitRatio=0.75)
df_Train <- subset(df, splitIndex == TRUE)
df_Test  <- subset(df, splitIndex == FALSE)

## Making the logistic regression model by using glm
logit <- glm(outcome ~ colname1 + colname2 + ..., 
             data=df_Train,
             family=binomial)
## We need to look for model with minimum AIC on the same training dataset
summary(logit)

## Making prediction based on Training set
predictTrain <- predict(logit, type="response")

## For making Confusion matrix 
table(df_Train$outcome, predictTrain > threshold)

## In order to choose best value for threshold we can plot the ROC curve
library(ROCR)
ROCRpred = prediction(predictTrain, df$outcome)
ROCRpref = performance(ROCRpred, "tpr", "fpr")
plot(ROCRpref, 
     colorize = TRUE, 
     print.cutoffs.at=seq(0,1,0.1), 
     text.adj=c(-0.2,1.7))

## For evaluating the quality of model we can check the value of AUC
# having AUC closer to 1.0 means we have better model
AUC = as.numeric(performance(ROCRpred, "auc")@y.values)

#### TIP: Selecting a range of independent variables for modeling
Sometime we need to select only a group of independent variables amoung a large number of variables in dataset which doing it with hand could be a tidious job. In order to achive that, we need to make a subset of variables which we want to use in model: 

In [ ]:
## First approach:
excluded_vars = c("colname1", "colname2", ...)
min_df <- df[ , !(names(df) %in% excluded_vars) ]
model <- glm(outcome~., data=min_df)

## Second approach (Just work on removing numeric variables):
model <- glm(outcome~.-colname1-colname2, data=df)

# Learning and classification
Generally there are two kind of classifier:
 - **Binary classification**
 - **Multiclass classification**

Most frequent algorithm for classification:
 - Support vector machines
 - Neural networks
 - Decision trees
 - NaÃve Bayes
 - Hidden Markov models
 
#### NaÃve Bayes
In this algorithm, we simply need to learn the probabilities by making the assumption that the attributes A and B **are independents**, hence the reason this model is defined as an independent feature model. NaÃve Bayes is widely used in text classification because the algorithm can be trained easily and efficiently. In NaÃve Bayes, we can calculate the probability of a condition A if B $(P(A|B))$, if you already know the probability of B given A $(P(B|A))$, as well as to A $(P(A))$ and B $(P(B))$ individually, as is shown in the previous Bayes Theorem example.

# Decision Trees

In [ ]:
library(rpart)
library(rpart.plot)
# Using rpart for list of explanatory variables for a discreate outcome
# class method (for ones and zeros output)
decisionTree <- rpart(Outcome ~ exp_var1 + exp_var2 + ...,
                      data=df_train,
                      method="class"#means use classification tree instead of regression tree,
                      minbucket=25 #determines the minimum number of observations in each bucket)
# To visualise the tree in screen
prp(decisionTree)

# If you wanted to predict a continuous variable as outcome
rpart(Outcome ~ exp_var1 + exp_var2 + ...,
               data=df_train,
               method="anova")

## Making prediction on test dataset
Prediction <- predict(decisionTree, 
                      newdata=df_test, 
                      type="class" #means giving the threshold of 0.5)
table(df_test$outcome, Prediction)

## To evaluate the model
library(ROCR)
PredictROC <- predict(decisionTree, newdata=df_test)
pred <- prediction(PredictROC[,2], df_test$outcome)
perf <- performance(pred, "tpr","fpr")
plot(perf)
# To get the AUC of the model
as.numeric(performance(pred, "auc")@y.values)

# Overfitting
Overfitting is technically defined as a model that performs better on a training set than another simpler model, but does worse on unseen data.

In general, we can set the `minbucket` value in rpart to define what should be the minimum number of observation in each bucket but the problem is the don't know this value. We can use the `k-fold croos-validation` technique to figure out that value.

In order to define this value in constructing model instead of minbucket we can set the `cp` value which mean **complexity parameter** which we can adjust the trade-off between model complexity and accuracy.

**NOTE:** Smaller cp leads to a bigger tree (might overfit).

In [ ]:
# First we need to load libraries
library(caret)
library(e1071)

# Then we need to define how many folds we want to have for cross-validation
numFolds <- trainControl(method="cv",
                         number=10)
# Then we need to define possible cp values by using expand.grid
cpGrid <- expand.grid(.cp=seq(0.01,0.5,0.01)) # or any other ranges
# Now, we are ready to perform cross validation
train(outcome ~ exp_var1 + exp_var2 + ..., 
      data=df_train,
      method="rpart",
      trControl=numFolds,
      tuneGrid=cpGrid)
# Now, we can construct our model by using cp instead of minbucket
rpart(Outcome ~ exp_var1 + exp_var2 + ...,
               data=df_train,
               method="class", 
               cp='TheOutcomeOfAboveCommand') #e.g. cp=0.18

# Prediction

Based on problem:
- If it's a classification problem, then we can use:
    * Logistic Regression
    * Naive Bayes
    * Decision Trees

## Ensemble
Take a large collection of individually imperfect models, and their one-off mistakes are probably not going to be made by the rest of them. If we average the results of all these models, we can sometimes find a superior model from their combination than any of the individual parts. That’s how ensemble models work, they grow a lot of different models, and let their outcomes be averaged or voted across the group.

### Random Forest
Random Forest models grow trees much deeper than the decision stumps above, in fact the default behaviour is to grow each tree out as far as possible. But since the formulas for building a single decision tree are the same every time, some source of randomness is required to make these trees different from one another. Random Forests do this in two ways.
 1. The first trick is to use bagging, for bootstrap aggregating. Bagging takes a randomized sample of the rows in your training set, with replacement.
 2. The second source of randomness gets past this limitation though. Instead of looking at the entire pool of available variables, Random Forests take only a subset of them, typically the square root of the number available. In our case we have 10 variables, so using a subset of three variables would be reasonable. The selection of available variables is changed for each and every node in the decision trees. 
 
R’s Random Forest algorithm has a few restrictions that we did not have with our decision trees. The big one has been the elephant in the room until now, we have to clean up the missing values in our dataset. 

In [ ]:
## Making an random forest model after data prepration process
# we used as.factor for outcome variable in order to make sure
# the output is same as classification
fit <- randomForest(as.factor(outcome) ~ exp_var1 + exp_var2 + ...,
                    data=df_train, 
                    importance=TRUE,
                    nodsize=25, #same as minbucket in CART
                    ntree=2000 #number of trees)

# To look at what variables were important:
 varImpPlot(fit)
             
## To make the prediction 
Prediction <- predict(fit, newdata=df_test)

There’s two types of importance measures shown above. The accuracy one tests to see how worse the model performs without each variable, so a high decrease in accuracy would be expected for very predictive variables. The Gini one digs into the mathematics behind decision trees, but essentially measures how pure the nodes are at the end of the tree. 

But let’s not give up yet. There’s more than one ensemble model. Let’s try a forest of conditional inference trees. They make their decisions in slightly different ways, using a statistical test rather than a purity measure, but the basic construction of each tree is fairly similar.

In [ ]:
install.packages('party')
library(party)

fit <- cforest(as.factor(outcome) ~ exp_var1 + exp_var2 + ...,
                 data = df_train, 
                 controls=cforest_unbiased(ntree=2000, mtry=3))

Prediction <- predict(fit, test, OOB=TRUE, type = "response")


# Text Analysis
The first thing for test mining is cleaning the text by doing some pre-processing operations.

In [ ]:
# Loading required libraries
library(tm)
library(SnowballC)

## First we need to construct corpus based on our data
corpus <- Corpus((VectorSource(df$text_colname)))

## Step1. Making the whole texts lowercase
corpus <- tm_map(corpus, tolower)

## Step2. Removing the puntuations
corpus <- tm_map(corpus, removePunctuation)

## Step3. Removing unhelpful words
corpus <- tm_map(corpus, 
                 removeWords, 
                 c("LIST_OF_ANY_WORD_YOU_WANT_TO_REMOVE", 
                   stopwords("english"))) # stopwords(english) is an internal dictionanry of unhelpful words

## Step4. Stem the document (it causes to mine the word roots)
corpus <- tm_map(corpus, stemDocument)

After doing the pre-processing steps on the text, now we are ready to count the words frequency which will be used in prediction model

In [ ]:
# From tm package we need to call below method to make a matrix of 
# documents as rows and words in each doc as columns
frequencies <- DocumentTermMatrix(corpus)

# In order to just take a look into a few rows and columns of frequencies we can do:
inspect(frequencies[1000:1005, 505:515])

# For having a list of most popular terms we can use:
findFreqTerms(frequencies, lowfreq=20) #lowfreq determines the minimum number of frequency

## Having to much terms as independent variable for making the prediction
# model couldn't be useful, so, we need to shorten the terms by removing non-frequent ones
sparse <- removeSparseTerms(frequencies, .995) #which means keep terms which are appear in more than 0.5 percent of all documents

# Now, we need to convert sparse matrix to data-frame which could be used for predicting model
docsSparse <- as.data.frame(as.matrix(sparse))
# And in order to make sure all columns names started with character:
colnames(docsSparse) = make.names(colnames(docsSparse))
# Then adding the outcome variable to newly generated dataframe
docsSparse$outcome <- df$outcome

## spliting the data for making the train/test set
split = sample.split(docsSparse$Negative, SplitRatio=.7)
trainSparse = subset(docsSparse, split==TRUE)
testSparse  = subset(docsSparse, split==FALSE)

Now, we prepared our data, we can use CART to predict our model. Which is not something new rather than normal prediction by using rpart and randomForest.

In [ ]:
library(rpart)
library(rpart.plot)
docCART = rpart(outcome ~ ., data=trainSparse, method="class")
# To take a look at tree
prp(tweetCART)
# making the prediction 
predictCART <- predict(docCART, newdata=testSparse,type="class")
# Checking the accuracy
table(testSparse$outcome, predictCART)
       predictCART7
        FALSE TRUE
  FALSE   294    6
  TRUE     37   18
# Accuracy: (294+18)/(294+6+37+18) ~= .87
# Compare to baseline
table(testSparse$outcome)
FALSE  TRUE 
  300    55
# Accuracy: (300)/(300+55) ~= .84 means the model does better than baseline

## making prediction by RandomForest
library(randomForest)
docRF <- randomForest(Negative ~ .,data=trainSparse)
predictRF = predict(docRF, newdata=testSparse)
# Checking the accuracy
table(testSparse$Negative, predictRF)
       predictRF
        FALSE TRUE
  FALSE   293    7
  TRUE     34   21
# Accuracy: (293+21)/(293+7+34+21) ~= .88 A bit better than CART but less interpretable